In [0]:
dbutils.widgets.removeAll()

In [0]:
#History
listening_history = []
artist_history = []
song_history = []

In [0]:
#Install Libraries
dbutils.library.installPyPI("pyviz")
dbutils.library.installPyPI("pyvis")
dbutils.library.installPyPI("networkx")
dbutils.library.installPyPI("grave")
dbutils.library.installPyPI("pandas")
dbutils.library.installPyPI("PyDictionary")

Out[3]: True

In [0]:
#Libraries
import string
#Search
from PyDictionary import PyDictionary 

#Graph Visualization
import networkx as nx
from networkx import drawing,layout
from pyvis.network import Network

#Graph Frames
from functools import reduce
import pyspark.sql.functions as F
from graphframes import *
from pyspark.sql.types import *


#Spotify
import json
import os
import requests

In [0]:
def build_network(graph_df,kind=None,num_songs=10,artist_name='David Byrne'):
  
  if not num_songs:
    num_songs = 400
  

  if kind == 'df_mb_db':

    selected_graph = graph_df.select('*').groupBy('work_id','hl_work_name').count().sort('count',ascending=False).limit(num_songs)

    to_graph = graph_df.join(selected_graph,['work_id','hl_work_name'],'inner')

    #[x for x in match_result.columns if 'artist' in x or x == 'valence' or x == 'energy']

  #if kind == 'artist' and isinstance(artist_name,str):

    #selected_graph = graph_df.where('hl_artist_name == "David Byrne"').drop('hl_artist_name')
    
    #Assemble conditions
    #work_id_list = []
    
    #for x in selected_graph.select('work_id').collect():
    #  work_id_list.append(x.work_id)
      
    #conditions = ' || workid == '.join(work_id_list)
    
    #Apply Coniditions
    #to_graph = graph_df.where(f'work_id == {conditions}')
    
  if kind is None:
    to_graph = graph_df

    #[x for x in match_result.columns if 'artist' in x or x == 'valence' or x == 'energy']

  vertices = to_graph.select("*")
  edges = to_graph.select("*")
    
   
  return vertices,edges
    


#def spotify_build_network(track_uri,artist_uri):
  
#  df = df_database.select("*").where(f'artist_uri == "{artist_uri}"')
    
def create_network(vertices,edges,ident,src,dst,save_file=None,kind=None):
  
  if kind == 'history':
    save_file = f'history_{save_file}.html'
  
  if kind == 'artist':
    save_file = f'artist_{save_file}.html'
    
  vertices = vertices.withColumnRenamed(ident,"id")
  edges = edges.withColumnRenamed(src,"src")
  edges = edges.withColumnRenamed(dst,"dst")

  #vertices = vertices.withColumnRenamed("hl_work_name","id")
  #edges = edges.withColumnRenamed("hl_work_name","src")
  #edges = edges.withColumnRenamed("hl_artist_name","dst")

  g = GraphFrame(vertices, edges)

  gp = nx.from_pandas_edgelist(g.edges.toPandas(),'src','dst',create_using=nx.MultiDiGraph())

  nt = Network("800px", "2000px")
  layout.spring_layout(gp,iterations=10,seed=1)
  nt.from_nx(gp)
  print(save_file)
  #Saves Graph
  nt.show(save_file)
  
  return g, str(save_file)

def run_html(file_name):
  with open(file_name, "r") as f: 
    displayHTML(f.read())
    

def join_frames(df_database,df_historylistens):
  
  df_database = df_database.withColumn("db_artist_track_name",F.concat(F.col("db_artist_name"),F.lit(' '),
  F.col("db_track_name")))

  df_historylistens = df_historylistens.withColumn("hl_artist_track_name",F.concat(F.col("hl_artist_name"),F.lit(' '),
    F.col("hl_work_name")))

  match_result = df_database.join(df_historylistens,df_database.db_artist_track_name == df_historylistens.hl_artist_track_name,'inner')


  return match_result
  

In [0]:
df_spotify_db = spark.read.csv("/FileStore/tables/adam_goldstein/MusicDataBase_SpotifyFeatures.csv",sep='|',header=True)

df_spotify_db = df_spotify_db.withColumnRenamed('artist_name', 'db_artist_name')
df_spotify_db = df_spotify_db.withColumnRenamed('track_name', 'db_track_name')


df_mb_db = spark.read.option("multiline","true").json("/FileStore/tables/adam_goldstein/Results.json")

df_mb_db = df_mb_db.withColumnRenamed('artist_name', 'hl_artist_name')
df_mb_db = df_mb_db.withColumnRenamed('work_name', 'hl_work_name')

df_mb_sdb = join_frames(df_spotify_db,df_mb_db)

dbutils.widgets.dropdown("Datasets",'df_mb_sdb', ['df_mb_sdb','df_spotify_db','df_mb_db'])

In [0]:
#Spotify Credentials
#Borrowed Outline of function from: https://github.com/TheComeUpCode/SpotifyGeneratePlaylist
user_name = "e820a33de9594a3ebb23e6eacb51421e"
token = "BQBJhvdgLOkAiUqXEvT5J8kue92EsrgwZt57aHMOt574nYUi-NwzELk6sZdtwj1hLyctT6plnTvVNvWWA1UxIKgXElkhhJDbIHIN9OJjcLu_x-TmFi4yBJ2TLedPaZIf72Hi5W7bfX57TjysdUzSRTt2oLuxyfm7yNtlykRhSD4aSd_6Ati_ibPbQE5IbZFYCbjIDTmazWwl4YyaSVH5FrDzGn-V7YHHGsEit58SrzAxw4HYvAN-z23a"

In [0]:
def clean(words,only_clean=None):
  new_list = []
  
  if only_clean:
    for word in words:
        modified = word.translate(str.maketrans('', '', string.punctuation))
        new_list.append(modified)
    return new_list
    

  for word in words:
    modified = word.translate(str.maketrans('', '', string.punctuation)) 
    added = f"'%{modified}%'"
    new_list.append(added)
  
  return new_list

import pandas as pd     
def temp_pandas(df):
  df = df.select("*").toPandas()
  return df

def back_to_spark(df):
  df = spark.createDataFrame(df)
  return df

def append_word(words,word):
  for w in words:
    if w in word:
      return w.capitalize()
            

In [0]:
def search(frame,criteria,label=None):
  
  if label == 'df_spotify_db':
    label = 'db_track_name'
  
  if label == 'df_mb_db':
    label = 'hl_work_name'
  
  if label == 'df_mb_sdb':
    label = 'hl_work_name'
  
  
  search_criteria = criteria[  : criteria.find(" ")]
    
  dictionary=PyDictionary() 
  sim_words_dirty = dictionary.synonym(f'{search_criteria}')
  
  if " " in search_criteria:
    sim_words_dirty = sim_words_dirty + search_criteria.split(" ")
  else:
    sim_words_dirty.append(search_criteria)
    
  sim_words = clean(sim_words_dirty)
    
  
  criteria = f' or {label} like '.join(sim_words) 
  #print(conditions)
  print(type(criteria))
  df = frame.select("*").filter(f"{label} like {criteria}")
  #df = frame.select("*").where(f"{label} like '%{search_criteria}%'")
  
  return df, clean(sim_words,only_clean=True)
  

    
  
  

In [0]:
#Endpoints
def get_current_playing(spotify_token,text=None):
    """Search For the Song"""
    query = "https://api.spotify.com/v1/me/player/currently-playing"
    response = requests.get(
        query,
        headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer {}".format(spotify_token)
        }
    )
    response_json = response.json()
    # check for valid response status
    if response.status_code != 200:
        print('Song is not playing')

    response_json = response.json()
      
    artist_uri = response_json['item']['artists'][0]['id']
    artist_name = response_json['item']['artists'][0]['name']
    track_url = response_json['item']['href']
    track_uri = response_json['item']['id']
    track_name = response_json['item']['name']
    
    listening_history.insert(0,f'{track_name} by {artist_name}')
    artist_history.insert(0,f'{artist_name}')
    song_history.insert(0,f'{track_name}')
    
    dbutils.widgets.dropdown("Music Listening History",f'{track_name} by {artist_name}', listening_history)
    dbutils.widgets.dropdown("Music Artist History",artist_name, artist_history)
    dbutils.widgets.dropdown("Music Song History", track_name, song_history)
  
  
def get_recently_played(spotify_token):
  
  query = "https://api.spotify.com/v1/me/player/currently-playing"
  response = requests.get(
    query,
    headers={
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(spotify_token)
    }
  )
  response_json = response.json()
  # check for valid response status
  if response.status_code != 200:
      print('Song is not playing')

  response_json = response.json()

  print(response_json)

  return artist_name,track_name,track_uri


In [0]:
get_current_playing(token)

df_search,sim_words = search(df_mb_sdb, dbutils.widgets.get("Music Song History"), label= dbutils.widgets.get("Datasets"))


temp = temp_pandas(df_search)


temp['word'] = temp.hl_work_name.apply(lambda x: append_word(sim_words,x))

df_search = back_to_spark(temp)

#display(df_search.select('*'))

#def build_network(graph_df,kind=None,num_songs=10,artist_name='David Byrne'):
vertices, edges = build_network(df_search)

vertices
#def create_network(vertices,edges,ident,src,dst,save_file=None,kind=None):
#'hl_work_name','hl_work_name','hl_artist_name'

plot_network, file = create_network(vertices,edges,'word','word','hl_artist_track_name',save_file='gx_presentation',kind='history')
print(sim_words)
run_html(file)

None 
 

<!-- 
-->